In [1]:
import pandas as pd
import numpy as np
import requests
import swifter
import re

In [26]:
# Read in 1.2 Million Spotify Songs dataset
spotify_1m = pd.read_csv('./Resources/tracks_features.csv')

In [27]:
# Strip brackets and single quotes from artist name
spotify_1m['artist'] = spotify_1m['artists'].str.strip("[']")

# Preview dataframe
spotify_1m.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,artist
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02,Rage Against The Machine
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02,Rage Against The Machine
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02,Rage Against The Machine
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02,Rage Against The Machine
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02,Rage Against The Machine


In [53]:
# Read in Billboard Top 100 dataset
df = pd.read_csv('./Resources/charts.csv')

# Convert date column to date type
df['date'] = pd.to_datetime(df['date'])
# Create new column for year
df['year'] = df['date'].dt.year
# Remove special characters from artist and song name
df['artist'] = df['artist'].str.replace(r"\(.*\)","")
# df['song'] = df['song'].str.replace(r"\(.*\)","")

df['song'] = df['song'].str.replace("\"", "") \
                        # .str.replace(",", "") \
                        # .str.replace("!", "")


# Create new column showing number of weeks in #1 spot (if exists)
# df['weeks-at-no1'] = df[df['rank']==1].groupby(['song', 'artist'], as_index=False).count()['date']

# Remove single quotes/apostrophes from song names
df['song'] = df['song'].str.replace("'", "")

# Preview dataframe
df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,2021
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,2021
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,2021


In [46]:
# Create new dataframe of all unique Billboard charting songs
unique_billboard_tracks_df = df.groupby(['song','artist', 'year'], as_index=False).agg({'peak-rank': 'min', 
                                                                                        'weeks-on-board': 'max',
                                                                                        'weeks-at-no1': 'max'})
# unique_billboard_tracks_df['weeks-at-no1'] = df[df['rank']==1].groupby(['song', 'artist'], as_index=False).count()['date']

# Create separate dataframes for each decade
billboard_1960s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1970) & (unique_billboard_tracks_df['year']>=1960)]
billboard_1970s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1980) & (unique_billboard_tracks_df['year']>=1970)]
billboard_1980s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1990) & (unique_billboard_tracks_df['year']>=1980)]
billboard_1990s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2000) & (unique_billboard_tracks_df['year']>=1990)]
billboard_2000s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2010) & (unique_billboard_tracks_df['year']>=2000)]
billboard_2010s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2020) & (unique_billboard_tracks_df['year']>=2010)]
billboard_2020s = unique_billboard_tracks_df[unique_billboard_tracks_df['year']>=2020]

# Create list of dataframes
billboard_dfs = [billboard_1960s, billboard_1970s, billboard_1980s, billboard_1990s, billboard_2000s, billboard_2010s, billboard_2020s]

# Display dataframe
unique_billboard_tracks_df.head()
unique_billboard_tracks_df[unique_billboard_tracks_df['weeks-at-no1'].isnull()==False]

# df[(df['artist']=='Beastie Boys') & (df['year']==1986)]

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1
388,2055,Sleepy Hallow,2021,51,15,14.0
399,24,Kanye West,2021,43,1,2.0
481,50 Shots,YoungBoy Never Broke Again,2021,59,1,5.0
488,5500 Degrees,"EST Gee Featuring Lil Baby, 42 Dugg & Rylo Rod...",2021,92,2,4.0
521,7am On Bridle Path,Drake,2021,16,3,2.0
...,...,...,...,...,...,...
35538,You Right,Doja Cat & The Weeknd,2021,11,18,7.0
35565,You Should Probably Leave,Chris Stapleton,2021,40,17,8.0
35599,You Time,Scotty McCreery,2021,50,12,14.0
35741,Your Heart,Joyner Lucas & J. Cole,2021,32,3,7.0


In [30]:
# SPOTIFY API
# Import dependencies for Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Import Client ID and Client Secret
from config import cid, secret


# Create objects for accessing Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [31]:
def search_spotify(song_title, artist):
    try:
        # print(f"Trying '{song_title}' by {artist}...")
        searchResults = sp.search(q=f"artist:{artist} track:{song_title}", type="track")
        track_id = searchResults['tracks']['items'][0]['id']
        # print(f"{song_title} by {artist} was successful!")
        # print(track_id)
        return track_id
    except:
        pass
        # print(f"'{song_title}' by {artist} returned 0 results.")

In [32]:
def adjust_parens(song_title):
    combinations = []

    strip_parens = ' '.join(song_title.strip(')(').split(')'))
    strip_parens = ' '.join(strip_parens.strip(' (').split('(')).replace('  ',' ')
    combinations.append(strip_parens)

    try:
        inside_parens = re.findall(r'\(.*?\)', song_title)[0].strip('()').strip()
    except IndexError:
        pass
        
    try:
        left_parens = song_title.split(f"({inside_parens})")[0].strip()
        if (left_parens not in combinations) & (len(left_parens) > 0): 
            combinations.append(left_parens)
    except:
        left_parens = "NA"

    try:
        right_parens = song_title.split(f"({inside_parens})")[1].strip()
        if (right_parens not in combinations) & (len(right_parens) > 0): 
            combinations.append(right_parens)
    except:
        right_parens = "NA"

    return combinations

In [33]:
# Create function to retrieve track ID from Spotify given the artist and song title
def get_track_id(song_title, artist):
    while True:
        # Select first artist if multiple listed with "Featuring" keyword
        if 'Featuring' in artist:
            artist = artist.split(' Featuring ')[0]
        # Select first artist if multiple listed with "with" keyword
        elif ' with ' in artist:
            artist = artist.split(' with ')[0]
        elif ' With ' in artist:
            artist = artist.split(' With ')[0]
        elif "," in artist:
            artist = artist.split(',')[0]

        # Make API call
        found_id = search_spotify(song_title, artist)
        if found_id:
            return found_id

        # Search artist and song title (replacing words ending in "in" to "ing")
        song_title = re.sub(r"in\b", 'ing ', song_title)
        found_id = search_spotify(song_title, artist)
        if found_id:
            return found_id

        if '(' in song_title:
            for item in adjust_parens(song_title):
                found_id = search_spotify(item, artist)
                if found_id:
                    return found_id
        # Check for '/' character in song_title
        if '/' in song_title:
            # Try string on left side of '/'
            song_title = song_title.split('/')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id

            # Try string on right side of '/'
            try:
                song_title = song_title.split('/')[1]
                found_id = search_spotify(song_title, artist)
                if found_id:
                    return found_id
            except:
                pass

        # Check for '&' character
        if ' & ' in artist:
            artist = artist.split(' & ')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id
        
        if ' X ' in artist:
            artist = artist.split(' X ')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id
            
        if ' x ' in artist:
            artist = artist.split(' x ')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id
            
        # Print song title and artist for non-match
        if found_id:
            return found_id
        
        break

            # print(f"No ID found for '{song_title}' by {artist}")
    

In [103]:
def get_audio_features(id):
    search_results = sp.audio_features(id)[0]
    danceability = search_results['danceability']
    energy = search_results['energy']
    key = search_results['key']
    loudness = search_results['loudness']
    mode = search_results['mode']
    speechiness = search_results['speechiness']
    acousticness = search_results['acousticness']
    instrumentalness = search_results['instrumentalness']
    liveness = search_results['liveness']
    valence = search_results['valence']
    tempo = search_results['tempo']
    duration_ms = search_results['duration_ms']
    time_signature = search_results['time_signature']

    return danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, \
                valence, tempo, duration_ms, time_signature

# get_audio_features('3MjUtNVVq3C8Fn0MP3zhXa')

In [76]:
song = 'One more time'
artist = "britney spears"

# result = sp.search(q=f"track:{song} artist:{artist} ", type="track")#['tracks']['items'][0]['id']

# result
# sp.search(q=f"track:{song} artist:{artist}", type='track', limit=1)
print(get_track_id(song, artist))
# adjust_parens(song)
    # search_spotify(song, artist)
# len(result)

# adjust_parens(song)

3MjUtNVVq3C8Fn0MP3zhXa


In [46]:
song_title = 'Life During Wartime (This Aint No PartyThis Aint No DiscoThis Aint)'
combinations = []

strip_parens = ' '.join(song_title.strip(')(').split(')'))
strip_parens = ' '.join(strip_parens.strip(' (').split('(')).replace('  ',' ')
combinations.append(strip_parens)

combinations

['Life During Wartime This Aint No PartyThis Aint No DiscoThis Aint', '']

In [42]:
# unique_billboard_tracks_df[unique_billboard_tracks_df['song']==song]

billboard_2020s[billboard_2020s['weeks-at-no1'].isnull()==False]

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1
388,2055,Sleepy Hallow,2021,51,15,14.0
399,24,Kanye West,2021,43,1,2.0
481,50 Shots,YoungBoy Never Broke Again,2021,59,1,5.0
488,5500 Degrees,"EST Gee Featuring Lil Baby, 42 Dugg & Rylo Rod...",2021,92,2,4.0
521,7am On Bridle Path,Drake,2021,16,3,2.0
...,...,...,...,...,...,...
35538,You Right,Doja Cat & The Weeknd,2021,11,18,7.0
35565,You Should Probably Leave,Chris Stapleton,2021,40,17,8.0
35599,You Time,Scotty McCreery,2021,50,12,14.0
35741,Your Heart,Joyner Lucas & J. Cole,2021,32,3,7.0


In [33]:
billboard_1960s

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1,track_id
10,(1-2-3-4-5-6-7) Count The Days,Inez & Charlie Foxx,1968,76,5,NaN,None
11,(A Ship Will Come) Ein Schiff Wird Kommen,Lale Anderson,1961,88,4,NaN,None
13,(All Of A Sudden) My Heart Sings,Mel Carter,1965,38,7,NaN,None
19,(Baby) Hully Gully,The Olympics,1960,72,7,NaN,None
29,(Come round Here) Im The One You Need,The Miracles,1966,17,9,NaN,None
...,...,...,...,...,...,...,...
36031,Zip Code,The Five Americans,1967,36,7,NaN,None
36032,Zip-A-Dee Doo-Dah,Bob B. Soxx And The Blue Jeans,1962,9,7,NaN,None
36033,Zip-A-Dee Doo-Dah,Bob B. Soxx And The Blue Jeans,1963,8,13,NaN,None
36039,Zorba The Greek,Herb Alpert & The Tijuana Brass,1965,82,1,NaN,None


In [11]:

billboard_1960s['track_id'] = billboard_1960s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1960s.to_csv(f'./Resources/billboard_1960s.csv', index=False)

Pandas Apply:   0%|          | 0/7792 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
billboard_1970s['track_id'] = billboard_1970s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1970s.to_csv(f'./Resources/billboard_1970s.csv', index=False)

Pandas Apply:   0%|          | 0/6251 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
billboard_1980s['track_id'] = billboard_1980s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1980s.to_csv(f'./Resources/billboard_1980s.csv', index=False)

Pandas Apply:   0%|          | 0/5126 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
billboard_1990s['track_id'] = billboard_1990s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1990s.to_csv(f'./Resources/billboard_1990s.csv', index=False)

Pandas Apply:   0%|          | 0/4405 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
billboard_2000s['track_id'] = billboard_2000s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2000s.to_csv(f'./Resources/billboard_2000s.csv', index=False)

Pandas Apply:   0%|          | 0/4433 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
billboard_2010s['track_id'] = billboard_2010s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2010s.to_csv(f'./Resources/billboard_2010s.csv', index=False)

Pandas Apply:   0%|          | 0/5582 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
billboard_2020s['track_id'] = billboard_2020s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2020s.to_csv(f'./Resources/billboard_2020s.csv', index=False)

Pandas Apply:   0%|          | 0/1479 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
df_1960s = pd.read_csv('./Resources/billboard_1960s.csv')
df_1970s = pd.read_csv('./Resources/billboard_1970s.csv')
df_1980s = pd.read_csv('./Resources/billboard_1980s.csv')
df_1990s = pd.read_csv('./Resources/billboard_1990s.csv')
df_2000s = pd.read_csv('./Resources/billboard_2000s.csv')
df_2010s = pd.read_csv('./Resources/billboard_2010s.csv')
df_2020s = pd.read_csv('./Resources/billboard_2020s.csv')

In [79]:
all_billboard_df = pd.concat([df_1960s, df_1970s, df_1980s, df_1990s, df_2000s, df_2010s, df_2020s])
all_billboard_df.to_csv('./Resources/billboard_ALL.csv', index=False)

In [69]:
all_billboard_df[all_billboard_df['track_id'].isnull()==False]

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1,track_id
0,(1-2-3-4-5-6-7) Count The Days,Inez & Charlie Foxx,1968,76,5,NaN,4wXNtDvLZl55vlQashWQrI
2,(All Of A Sudden) My Heart Sings,Mel Carter,1965,38,7,NaN,6x5BiQwNlbtisITsEHa8Eu
3,(Baby) Hully Gully,The Olympics,1960,72,7,NaN,2CkbNJooPxUsHG4gFXKhxa
4,(Come round Here) Im The One You Need,The Miracles,1966,17,9,NaN,5YvokzRkNONpj5c0TIqTjQ
5,(Dance The) Mess Around,Chubby Checker,1961,24,7,NaN,58EirkLFB6O6wwRN9JowHB
...,...,...,...,...,...,...,...
1467,homecoming queen?,Kelsea Ballerini,2020,65,13,NaN,1ytCra0qH6gcHnCVQwREvu
1469,ily,Surf Mesa Featuring Emilee,2020,23,28,NaN,62aP9fBQKYKxi7PDXwcUAS
1470,interlude,J. Cole,2021,8,3,NaN,2NJu5JtZLPmmuZF7Lr4Arr
1473,pov,Ariana Grande,2020,40,6,NaN,3UoULw70kMsiVXxW0L3A33


In [74]:
x = all_billboard_df.merge(spotify_1m, how="left", left_on="track_id", right_on="id")
x[x['id'].isnull()==False]

,song,artist_x,year_x,peak-rank,weeks-on-board,weeks-at-no1,track_id,id,name,album,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year_y,release_date,artist_y
21,(I Cant Help You) Im Falling Too,Skeeter Davis,1960,39,7,NaN,5MXe6dKcR6iO33GWzVW8cr,5MXe6dKcR6iO33GWzVW8cr,(I Can't Help You) I'm Falling Too,The Essential Skeeter Davis,...,0.7770,0.000000,0.1330,0.3840,91.209,165267.0,4.0,1994.0,1994-05-23,Skeeter Davis
66,(Sweet Sweet Baby) Since Youve Been Gone,Aretha Franklin,1968,5,12,NaN,3lrSnngQqNWgvQmcFbCJCx,3lrSnngQqNWgvQmcFbCJCx,(Sweet Sweet Baby) Since You've Been Gone,Lady Soul (With Bonus Selections),...,0.4710,0.031400,0.0631,0.9400,117.149,145507.0,4.0,1968.0,1968-01-22,Aretha Franklin
72,(The Lights Went Out In) Massachusetts,Bee Gees,1967,11,8,NaN,33YKKW1MaDUwq7GDw3WOqR,33YKKW1MaDUwq7GDw3WOqR,[The Lights Went Out In] Massachusetts/Words/H...,Rock and Roll Hall of Fame Volume 4: 1996- 1997,...,0.1490,0.000680,0.5510,0.3810,104.050,176320.0,4.0,2011.0,2011-11-15,Bee Gees
80,(Theme From) Valley Of The Dolls,Dionne Warwick,1968,2,13,NaN,021O8Fvrww2EJB8220tgXo,021O8Fvrww2EJB8220tgXo,(Theme From) Valley of the Dolls,The Valley Of The Dolls,...,0.8710,0.000002,0.0590,0.0868,74.919,218840.0,4.0,1968.0,1968,Dionne Warwick
84,(Theme from) The Great Impostor,Henry Mancini And His Orchestra,1961,90,1,NaN,6bXt2g5gzKiM7uZ4RKgF6X,6bXt2g5gzKiM7uZ4RKgF6X,"Theme From ""The Great Impostor"" - REMASTERED",Greatest Hits - The Best of Henry Mancini,...,0.3840,0.164000,0.3610,0.4150,86.833,164304.0,4.0,2000.0,2000-09-14,Henry Mancini
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35032,You Broke Me First.,Tate McRae,2021,17,38,NaN,45bE4HXI0AwGZXfZtMp8JR,45bE4HXI0AwGZXfZtMp8JR,you broke me first,you broke me first,...,0.7860,0.000000,0.0906,0.0799,124.099,169266.0,4.0,2020.0,2020-04-17,Tate McRae
35034,You Got It,VEDO,2020,87,4,NaN,4lsHZ92XCFOQfzJFBTluk8,4lsHZ92XCFOQfzJFBTluk8,You Got It,For You,...,0.2970,0.000000,0.1650,0.3910,122.089,201587.0,4.0,2020.0,2020-03-27,Vedo
35035,You Got It,VEDO,2021,75,16,NaN,4lsHZ92XCFOQfzJFBTluk8,4lsHZ92XCFOQfzJFBTluk8,You Got It,For You,...,0.2970,0.000000,0.1650,0.3910,122.089,201587.0,4.0,2020.0,2020-03-27,Vedo
35051,Zoo York,Lil Tjay Featuring Fivio Foreign & Pop Smoke,2020,65,1,NaN,3Ddny34bFoYBb8dIGq5lwp,3Ddny34bFoYBb8dIGq5lwp,Zoo York (feat. Fivio Foreign & Pop Smoke),State of Emergency,...,0.0352,0.000009,0.1060,0.3030,92.466,214825.0,5.0,2020.0,2020-05-08,"Lil Tjay', 'Fivio Foreign', 'Pop Smoke"


In [105]:
print(get_audio_features("2dNhNlpZAtV4lwAFW45LIv"))

df_2020s.dropna(subset=['track_id'], inplace=True)


# x[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
#                  'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']] = x\
#                     .swifter.apply(lambda row:get_audio_features(row.track_id),axis=1)
# x.to_csv(f'./Resources/billboard_1960s_features.csv', index=False)

df_2020s['danceability'], df_2020s['energy'], df_2020s['key'], df_2020s['loudness'], df_2020s['mode'], df_2020s['speechiness'], df_2020s['acousticness'], \
    df_2020s['instrumentalness'], df_2020s['liveness'], df_2020s['valence'], df_2020s['tempo'], df_2020s['duration_ms'], df_2020s['time_signature'] = \
                                          zip(*df_2020s['track_id'].map(get_audio_features))

df_2020s.to_csv(f'./Resources/billboard_2020s_features.csv', index=False)

(0.83, 0.517, 6, -6.416, 0, 0.333, 0.399, 8.95e-05, 0.0916, 0.616, 171.959, 182496, 4)


In [108]:
df_2010s.dropna(subset=['track_id'], inplace=True)

df_2010s['danceability'], df_2010s['energy'], df_2010s['key'], df_2010s['loudness'], df_2010s['mode'], df_2010s['speechiness'], df_2010s['acousticness'], \
    df_2010s['instrumentalness'], df_2010s['liveness'], df_2010s['valence'], df_2010s['tempo'], df_2010s['duration_ms'], df_2010s['time_signature'] = \
                                          zip(*df_2010s['track_id'].map(get_audio_features))
df_2010s.to_csv(f'./Resources/billboard_2010s_features.csv', index=False)

In [85]:
df_2000s.dropna(subset=['track_id'], inplace=True)

df_2000s['danceability'], df_2000s['energy'], df_2000s['key'], df_2000s['loudness'], df_2000s['mode'], df_2000s['speechiness'], df_2000s['acousticness'], \
    df_2000s['instrumentalness'], df_2000s['liveness'], df_2000s['valence'], df_2000s['tempo'], df_2000s['duration_ms'], df_2000s['time_signature'] = \
                                          zip(*df_2000s['track_id'].map(get_audio_features))
df_2000s.to_csv(f'./Resources/billboard_2000s_features.csv', index=False)

In [ ]:
df_1990s.dropna(subset=['track_id'], inplace=True)

df_1990s['danceability'], df_1990s['energy'], df_1990s['key'], df_1990s['loudness'], df_1990s['mode'], df_1990s['speechiness'], df_1990s['acousticness'], \
    df_1990s['instrumentalness'], df_1990s['liveness'], df_1990s['valence'], df_1990s['tempo'], df_1990s['duration_ms'], df_1990s['time_signature'] = \
                                          zip(*df_1990s['track_id'].map(get_audio_features))
df_1990s.to_csv(f'./Resources/billboard_1990s_features.csv', index=False)

In [ ]:
df_1980s.dropna(subset=['track_id'], inplace=True)

df_1980s['danceability'], df_1980s['energy'], df_1980s['key'], df_1980s['loudness'], df_1980s['mode'], df_1980s['speechiness'], df_1980s['acousticness'], \
    df_1980s['instrumentalness'], df_1980s['liveness'], df_1980s['valence'], df_1980s['tempo'], df_1980s['duration_ms'], df_1980s['time_signature'] = \
                                          zip(*df_1980s['track_id'].map(get_audio_features))
df_1980s.to_csv(f'./Resources/billboard_1980s_features.csv', index=False)

In [ ]:
df_1970s.dropna(subset=['track_id'], inplace=True)

df_1970s['danceability'], df_1970s['energy'], df_1970s['key'], df_1970s['loudness'], df_1970s['mode'], df_1970s['speechiness'], df_1970s['acousticness'], \
    df_1970s['instrumentalness'], df_1970s['liveness'], df_1970s['valence'], df_1970s['tempo'], df_1970s['duration_ms'], df_1970s['time_signature'] = \
                                          zip(*df_1970s['track_id'].map(get_audio_features))
df_1970s.to_csv(f'./Resources/billboard_1970s_features.csv', index=False)

In [ ]:
df_1960s.dropna(subset=['track_id'], inplace=True)

df_1960s['danceability'], df_1960s['energy'], df_1960s['key'], df_1960s['loudness'], df_1960s['mode'], df_1960s['speechiness'], df_1960s['acousticness'], \
    df_1960s['instrumentalness'], df_1960s['liveness'], df_1960s['valence'], df_1960s['tempo'], df_1960s['duration_ms'], df_1960s['time_signature'] = \
                                          zip(*df_1960s['track_id'].map(get_audio_features))
df_1960s.to_csv(f'./Resources/billboard_1960s_features.csv', index=False)